# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In [1]:
# Dependencies and Setup

import pandas as pd
import requests


In [2]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [3]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file
import json

file_path = "C_intensity_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(responses_dict, json_file, indent=4)

print("Dictionary successfully exported to JSON file.")

df_carbon_intensity_history = pd.read_json(file_path)


Dictionary successfully exported to JSON file.


In [4]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file

file_path = "power_breakdown_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(power_breakdown_responses_dict, json_file, indent=4)

df_power_breakdown_history = pd.read_json(file_path)

SECTION 2     
PowerBreakdown data transformation

In [5]:
# pull data from power breakdown json in dataframe
region = df_power_breakdown_history['response_1']['history'][0]['zone']
datetime = df_power_breakdown_history['response_1']['history'][0]['datetime']
nuclear = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = df_power_breakdown_history['response_1']['history'][0]["renewablePercentage"]
total_consumption = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionTotal"]
estimated = df_power_breakdown_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1 = {'region':region,'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}

# Create a dataFrame with the first values
df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [6]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_power_breakdown_history[f"{response}"]['history'][i]['zone']
        datetime = df_power_breakdown_history[f"{response}"]['history'][i]['datetime']
        nuclear = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = df_power_breakdown_history[f"{response}"]['history'][i]["renewablePercentage"]
        total_consumption = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionTotal"]
        estimated = df_power_breakdown_history[f"{response}"]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}
#set up the times as rows and measurements as columns
df_US_new = df_US.transpose()

# check data types
df_US_new.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_37124\3249227210.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US[24*reg+i]= {'region':region, 'datetime':datetime,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar,


,region,datetime,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated
count,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1343,1344,1344
unique,53,24,288,23,59,547,406,401,539,920,44,296,5,31,85,1112,2
top,US-CAL-BANC,2024-07-18T22:00:00.000Z,0,0,0,0,0,0,0,0,0,0,0,0,0,74,True
freq,48,56,818,1320,1217,618,724,690,339,192,1200,564,1340,1300,197,7,847


In [7]:
# fill NA values with zeroes for energy values
df_US_new = df_US_new.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 
           'battery_discharge':0})
df_US_new.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.00000,1344.000000,1344.000000
mean,1059.023065,13.401786,19.855655,1427.258185,546.194940,487.440476,567.776786,3636.023065,1.579613,99.66369,0.316964,15.363839
std,2414.356989,99.553534,100.355355,3577.430639,1733.163565,1950.125925,1212.066109,7354.950021,6.930754,211.65226,7.388233,238.105314
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,277.750000,0.000000,0.00000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,48.000000,0.000000,0.000000,69.000000,1035.000000,0.000000,9.00000,0.000000,0.000000
75%,325.000000,0.000000,0.000000,1236.000000,176.000000,173.250000,665.750000,2405.250000,0.000000,94.25000,0.000000,0.000000
max,13732.000000,790.000000,614.000000,26978.000000,14556.000000,16950.000000,9834.000000,44305.000000,81.000000,1789.00000,226.000000,5526.000000


In [8]:
# convert measured Energy values to integers in Giga Watts
convert_dict = {'hydro': int}
#convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
             #   'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
               # }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US_new.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

region                  object
datetime                object
nuclear                  int64
geothermal               int64
biomass                  int64
coal                     int64
wind                     int64
solar                    int64
hydro                    int32
gas                      int64
oil                      int64
unknown                  int64
hydro-discharge          int64
battery_discharge        int64
renewable_percentage    object
total_consumption       object
estimated               object
dtype: object

In [9]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime

In [10]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop('datetime', axis=1)
df_US_newer.head()

,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date
UTC DateTime,,,,,,,,,,,,,,,,,,
2024-07-18 22:00:00,US-SW-PNM,0,0,0,360,313,660,17,683,0,0,0,0,49,2033,False,22:00:00,2024-07-18
2024-07-18 23:00:00,US-SW-PNM,0,0,0,409,676,517,17,680,0,0,0,0,53,2299,False,23:00:00,2024-07-18
2024-07-19 00:00:00,US-SW-PNM,0,0,0,432,1058,316,17,674,0,12,0,0,55,2509,False,00:00:00,2024-07-19
2024-07-19 01:00:00,US-SW-PNM,0,0,0,482,1189,93,17,690,0,195,0,0,49,2666,False,01:00:00,2024-07-19
2024-07-19 02:00:00,US-SW-PNM,0,0,0,502,780,2,17,702,0,209,0,0,36,2212,False,02:00:00,2024-07-19


Section 3
Transform carbon intensity data

In [11]:
# pull data from C intensity json in dataframe
region = df_carbon_intensity_history['response_1']['history'][0]['zone']
datetime = df_carbon_intensity_history['response_1']['history'][0]['datetime']
carbon_Intensity = df_carbon_intensity_history['response_1']['history'][0]["carbonIntensity"]
estimated = df_carbon_intensity_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1C = {'region':region,'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_US_C = pd.DataFrame.from_dict(us_pnm1C,orient='index')

In [12]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_carbon_intensity_history[f"{response}"]['history'][i]['zone']
        datetime = df_carbon_intensity_history[f"{response}"]['history'][i]['datetime']
        carbon_Intensity = df_carbon_intensity_history['response_1']['history'][i]["carbonIntensity"]
        estimated = df_carbon_intensity_history['response_1']['history'][i]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_US_C_new = df_US_C.transpose()

# check data types
df_US_C_new.dtypes

C:\Users\mrkol\AppData\Local\Temp\ipykernel_37124\3332989139.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US_C[24*reg+i]= {'region':region, 'datetime':datetime,'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}


region              object
datetime            object
Carbon_Intensity    object
estimated           object
dtype: object

In [13]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_US_C_new = df_US_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_US_C_new.dtypes

region              object
datetime            object
Carbon_Intensity     int32
estimated           object
dtype: object

In [14]:
# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_US_C_new['UTC time'] = times
df_US_C_new['UTC date'] = dates
df_US_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_US_C_new_reindex = df_US_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_US_C_newer = df_US_C_new.drop('datetime', axis=1)

Section 4
Merge dataframes

In [15]:
df_power_and_carbon= pd.merge(df_US_newer, df_US_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,
2024-07-18 22:00:00,US-SW-PNM,0,0,0,360,313,660,17,683,0,0,0,0,49,2033,False,22:00:00,2024-07-18,412,False
2024-07-18 23:00:00,US-SW-PNM,0,0,0,409,676,517,17,680,0,0,0,0,53,2299,False,23:00:00,2024-07-18,390,False
2024-07-19 00:00:00,US-SW-PNM,0,0,0,432,1058,316,17,674,0,12,0,0,55,2509,False,00:00:00,2024-07-19,371,False
2024-07-19 01:00:00,US-SW-PNM,0,0,0,482,1189,93,17,690,0,195,0,0,49,2666,False,01:00:00,2024-07-19,422,False
2024-07-19 02:00:00,US-SW-PNM,0,0,0,502,780,2,17,702,0,209,0,0,36,2212,False,02:00:00,2024-07-19,524,False


In [16]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7513.000000,7535.000000,7572.000000
mean,1252.368859,15.735605,25.357501,1779.768753,698.259905,554.066561,689.263074,4355.473455,10.308241,100.329371,0.631141,19.627839,32.702383,9547.940677,337.720153
std,2566.567950,108.035557,113.886265,4499.025480,2390.787580,2105.934267,1458.410363,8453.829766,69.590301,208.795902,13.341312,284.931901,34.239063,17497.795423,153.621494
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,235.750000,0.000000,0.000000,0.000000,0.000000,5.000000,910.000000,225.000000
50%,0.000000,0.000000,0.000000,44.000000,0.000000,1.000000,73.000000,1000.500000,0.000000,10.000000,0.000000,0.000000,20.000000,2411.000000,310.000000
75%,1029.500000,0.000000,0.000000,1394.000000,224.000000,217.000000,762.250000,3699.750000,0.000000,96.000000,0.000000,0.000000,50.000000,9635.000000,408.000000
max,13803.000000,812.000000,657.000000,34899.000000,21642.000000,19768.000000,11538.000000,54713.000000,2054.000000,1864.000000,426.000000,7075.000000,100.000000,117668.000000,692.000000


In [17]:
df_us_energy_reindex.drop_duplicates(inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption(GW),Carbon_Intensity(gCO2eq/kWh)
count,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7572.000000,7513.000000,7535.000000,7572.000000
mean,1252.368859,15.735605,25.357501,1779.768753,698.259905,554.066561,689.263074,4355.473455,10.308241,100.329371,0.631141,19.627839,32.702383,9547.940677,337.720153
std,2566.567950,108.035557,113.886265,4499.025480,2390.787580,2105.934267,1458.410363,8453.829766,69.590301,208.795902,13.341312,284.931901,34.239063,17497.795423,153.621494
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,235.750000,0.000000,0.000000,0.000000,0.000000,5.000000,910.000000,225.000000
50%,0.000000,0.000000,0.000000,44.000000,0.000000,1.000000,73.000000,1000.500000,0.000000,10.000000,0.000000,0.000000,20.000000,2411.000000,310.000000
75%,1029.500000,0.000000,0.000000,1394.000000,224.000000,217.000000,762.250000,3699.750000,0.000000,96.000000,0.000000,0.000000,50.000000,9635.000000,408.000000
max,13803.000000,812.000000,657.000000,34899.000000,21642.000000,19768.000000,11538.000000,54713.000000,2054.000000,1864.000000,426.000000,7075.000000,100.000000,117668.000000,692.000000


In [18]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_power_and_carbon])

# drop duplicate rows
df_both.drop_duplicates(inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,Carbon_Intensity(gCO2eq/kWh)
count,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000,8844.000000
mean,1233.128223,15.509046,24.727838,1733.377883,679.968114,547.038783,673.384102,4270.184645,9.065694,100.788218,0.588535,19.139643,359.783808
std,2552.977384,107.222860,112.396300,4390.904811,2313.030362,2091.261109,1429.324531,8330.826616,64.518782,209.883359,12.676276,279.495566,158.045459
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.750000,0.000000,0.000000,0.000000,0.000000,230.750000
50%,0.000000,0.000000,0.000000,42.000000,0.000000,1.000000,73.000000,999.000000,0.000000,10.000000,0.000000,0.000000,333.000000
75%,959.500000,0.000000,0.000000,1387.000000,215.000000,213.000000,738.000000,3464.750000,0.000000,96.000000,0.000000,0.000000,445.000000
max,13803.000000,812.000000,657.000000,34899.000000,21642.000000,19768.000000,11538.000000,54713.000000,2054.000000,1864.000000,426.000000,7075.000000,692.000000


In [19]:
#df_both.to_csv(f'data/{dates[0]}USenergy_data2.csv')
df_both.to_csv(f'data/runningUSenergy_data.csv')

In [20]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,hydro-discharge,battery_discharge,Carbon_Intensity(gCO2eq/kWh)
count,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000,3090.000000
mean,1317.785113,42.430744,64.183172,1923.558900,1107.975081,720.016181,1057.892880,5343.009061,19.316828,117.863107,0.000647,52.408414,350.468285
std,2278.452454,174.120281,173.604637,4715.671007,2937.253518,2663.912523,1694.253952,8540.105411,98.458062,230.874478,0.025437,465.275603,124.994476
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,113.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.250000,237.250000,0.000000,0.000000,0.000000,0.000000,250.000000
50%,0.000000,0.000000,0.000000,34.000000,31.000000,1.000000,215.000000,1286.000000,0.000000,13.000000,0.000000,0.000000,333.000000
75%,2166.500000,0.000000,0.000000,1296.000000,463.000000,220.500000,1325.000000,8515.500000,0.000000,128.000000,0.000000,0.000000,422.000000
max,13287.000000,812.000000,657.000000,34149.000000,21642.000000,19768.000000,10136.000000,54713.000000,2054.000000,1864.000000,1.000000,7075.000000,692.000000


In [21]:
df_both_cleaned.to_csv(f'data/runningUSenergy_data_filtered.csv')